<a href="https://colab.research.google.com/github/blelliott23/neuro-data-analysis-course/blob/master/VDR_Decoding_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5, style="whitegrid", palette="pastel", color_codes=True)

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_predict, LeaveOneOut
import os
import csv

In [2]:
##Download data file, set name

!wget https://github.com/blelliott23/neuro-data-analysis-course/blob/master/VDR_Decode/Data/136/136_Data_Final.csv?raw=true
!mv 136_Data_Final.csv\?raw\=true 136_Data_Final.csv

--2019-04-17 08:21:13--  https://github.com/blelliott23/neuro-data-analysis-course/blob/master/VDR_Decode/Data/136/136_Data_Final.csv?raw=true
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/blelliott23/neuro-data-analysis-course/raw/master/VDR_Decode/Data/136/136_Data_Final.csv [following]
--2019-04-17 08:21:13--  https://github.com/blelliott23/neuro-data-analysis-course/raw/master/VDR_Decode/Data/136/136_Data_Final.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/blelliott23/neuro-data-analysis-course/master/VDR_Decode/Data/136/136_Data_Final.csv [following]
--2019-04-17 08:21:13--  https://raw.githubusercontent.com/blelliott23/neuro-data-analysis-course/master/VDR_Decode/Data/136/136_Data_Final.csv
Resolving raw.githubus

In [3]:
#Import data to dataframe
##Implement Random forest stuff here...
df = pd.read_csv('136_Data_Final.csv')
df.set_index('Subject')
df.head()

,Unnamed: 0,Subject,Block,Trial,Time,O2,O1,OZ,PZ,P4,...,FC3,F3,FP2,F7,FP1,VEOU,VEOL,HEOR,HEOL,Value
0,0,136,1,1,-200,-2.6587,-5.8283,-3.6200,-8.4912,-6.0398,...,-4.2465,-6.1961,-0.7449,-4.0477,-6.5453,-252.5419,70.5294,13.6076,-66.9374,1.0
1,1,136,1,1,-196,-3.3376,-5.6611,-3.5900,-8.8003,-6.9963,...,-3.2880,-5.5053,-0.7434,-3.1528,-4.8430,-218.5595,51.3878,25.2795,-68.4394,1.0
2,2,136,1,1,-192,-3.4338,-4.6322,-2.9857,-8.3836,-7.6261,...,-2.0604,-4.9363,-0.8046,-2.6793,-2.9363,-196.0244,68.5147,18.5178,-64.9081,1.0
3,3,136,1,1,-188,-3.6411,-3.6174,-2.5697,-7.9997,-8.4763,...,-1.1900,-5.0391,-0.8101,-2.7765,-1.7893,-191.9580,51.4932,22.3986,-66.2655,1.0
4,4,136,1,1,-184,-3.9098,-3.0866,-2.5453,-8.4382,-9.8238,...,-0.9082,-5.5007,-0.6975,-3.0212,-2.2171,-177.3466,41.4210,21.8185,-66.5214,1.0


In [0]:
#df_one_hot_encoded = pd.get_dummies(df)
#df = df_one_hot_encoded


In [26]:
df_1 = df.groupby(np.arange(len(df))//15).mean()
df_1['High Value'] = 0
df_1.loc[df_1['Value'] >5, 'High Value'] = 1

df_1.head(60)

,Unnamed: 0,Subject,Block,Trial,Time,O2,O1,OZ,PZ,P4,...,F3,FP2,F7,FP1,VEOU,VEOL,HEOR,HEOL,Value,High Value
0,7.0,136.0,1.0,1.0,-172.0,-3.816187,0.086460,-1.681933,-7.700933,-9.183987,...,-4.000933,-1.955880,-2.511913,-3.581907,-81.870200,-3.303380,22.142160,-47.071013,1.0,0
1,22.0,136.0,1.0,1.0,-112.0,5.041507,2.270940,3.706127,5.861633,6.079707,...,3.175580,4.078860,2.596980,3.405900,159.992167,-74.789260,5.412680,29.089273,1.0,0
2,37.0,136.0,1.0,1.0,-52.0,2.076793,-0.253500,0.707507,4.291613,6.048160,...,-1.323787,-2.720907,-0.196007,0.261760,-33.129200,44.762173,-20.111080,17.082507,1.0,0
3,52.0,136.0,1.0,1.0,8.0,-12.334533,-11.674047,-11.438907,-6.048700,-7.308500,...,-0.029273,-2.851647,-3.633487,-3.036847,-173.019793,110.445047,-34.240380,-2.448473,1.0,0
4,67.0,136.0,1.0,1.0,68.0,-9.983780,-2.166600,-5.763527,4.783927,-2.761300,...,-3.648467,-10.245800,-10.307447,-9.395833,-229.155360,135.185473,-48.530300,-9.114373,1.0,0
5,82.0,136.0,1.0,1.0,128.0,-7.925880,-2.490533,-5.297840,-2.093733,-6.615940,...,-11.610900,-14.606560,-16.310327,-12.936940,-249.846520,145.647847,-42.216487,-18.075567,1.0,0
6,97.0,136.0,1.0,1.0,188.0,-6.561920,-0.099200,-3.151360,6.523360,1.104460,...,-0.761020,-3.539193,-6.173327,-0.092287,-252.230500,155.437627,-33.929767,-15.257733,1.0,0
7,112.0,136.0,1.0,1.0,248.0,-11.624793,-6.961573,-10.789220,-3.378507,-5.692373,...,0.164780,-2.728340,-3.866513,-0.302153,-255.760453,160.001487,-14.289020,-20.692500,1.0,0
8,127.0,136.0,1.0,1.0,308.0,-7.746087,0.816853,-6.629307,-9.190240,-8.376440,...,-4.372373,-7.938513,-6.688907,-4.526947,-267.736107,157.263547,-16.379380,-29.454473,1.0,0
9,142.0,136.0,1.0,1.0,368.0,-5.105160,9.705167,0.810820,1.454380,-2.008367,...,0.153973,-9.205407,-8.031647,-5.404973,-280.407967,154.626820,-26.580293,-32.599920,1.0,0


In [5]:
import random
random_samp_list = []
index_list = []
for i in range(1,201):
  index_list.append(i)


train_samp_list = random.sample(index_list,k=100)
  
test_list = []
test_list = [x for x in index_list if x not in train_samp_list]


#df_train = df['Trial'] == i
print(train_samp_list)
print(test_list)

[122, 151, 72, 137, 45, 96, 6, 23, 92, 194, 7, 91, 36, 144, 19, 62, 157, 198, 189, 134, 35, 187, 173, 174, 94, 10, 65, 114, 105, 141, 102, 138, 34, 128, 130, 110, 150, 183, 31, 33, 163, 169, 111, 98, 140, 3, 54, 191, 190, 129, 61, 40, 132, 145, 171, 172, 80, 41, 135, 165, 185, 162, 131, 116, 46, 107, 146, 25, 50, 78, 30, 76, 152, 5, 86, 97, 56, 149, 118, 142, 58, 168, 166, 83, 101, 99, 133, 17, 47, 124, 119, 44, 77, 170, 158, 200, 126, 155, 55, 81]
[1, 2, 4, 8, 9, 11, 12, 13, 14, 15, 16, 18, 20, 21, 22, 24, 26, 27, 28, 29, 32, 37, 38, 39, 42, 43, 48, 49, 51, 52, 53, 57, 59, 60, 63, 64, 66, 67, 68, 69, 70, 71, 73, 74, 75, 79, 82, 84, 85, 87, 88, 89, 90, 93, 95, 100, 103, 104, 106, 108, 109, 112, 113, 115, 117, 120, 121, 123, 125, 127, 136, 139, 143, 147, 148, 153, 154, 156, 159, 160, 161, 164, 167, 175, 176, 177, 178, 179, 180, 181, 182, 184, 186, 188, 192, 193, 195, 196, 197, 199]


In [0]:
rfc = RandomForestClassifier(n_estimators=1000, max_features='auto')
#rfc.fit(x_pd, y_pd);

In [31]:
#df_1 = df_1.drop(['Unnamed: 0','Subject'], axis=1)
df_2

,Block,Trial,Time,O2,O1,OZ,PZ,P4,CP4,P8,...,F4,F8,T7,FT7,FC3,F3,FP2,F7,FP1,High Value
10,1.0,1.000000,428.000000,-15.426907,2.694200,-9.115193,-13.547847,-14.755420,-14.297733,-12.203133,...,-12.324127,-12.327933,-4.800553,-8.921300,-10.157213,-9.283707,-10.847200,-9.377980,-6.324813,0
11,1.0,1.000000,488.000000,-12.743320,6.769787,-5.430427,-10.975547,-12.776020,-13.578220,-9.524653,...,-13.583907,-10.466533,-4.770047,-6.765160,-7.526493,-9.071667,-10.571807,-7.160487,-7.216447,0
12,1.0,1.000000,548.000000,-24.480247,-1.283380,-15.356733,-17.816287,-20.714667,-16.379140,-20.879853,...,5.304567,-2.555947,-0.489700,4.161640,-2.316567,5.565427,4.594340,3.685287,5.559540,0
13,1.0,1.000000,608.000000,-20.349313,2.116753,-12.485680,-16.179287,-19.548360,-16.741853,-18.277247,...,-12.094100,-11.762393,-1.843087,-4.703280,-4.177633,-3.338733,-7.848207,-5.201220,-3.404273,0
14,1.0,1.000000,668.000000,-20.823787,2.815493,-11.740127,-12.132240,-16.717380,-14.018593,-17.954833,...,-1.999753,-8.921720,-0.028287,-1.991227,-3.164027,-1.846907,-1.450467,-2.459240,-1.816820,0
15,1.0,1.000000,728.000000,-17.263307,2.576473,-10.995153,-12.644787,-12.564807,-9.304280,-13.532973,...,0.863900,-5.090087,-5.417233,-3.394687,-3.601767,0.363940,1.128580,-3.883313,0.186920,0
16,1.0,1.000000,788.000000,-30.572640,-4.302940,-21.136053,-18.610560,-23.626473,-19.416313,-25.247220,...,-8.402880,-5.073553,0.736973,0.643400,0.059520,-2.207927,-2.557273,0.236413,1.576367,0
47,1.0,2.000000,448.000000,-14.003793,2.640647,-8.252713,-20.717860,-23.719660,-23.072873,-13.867613,...,-19.141167,-12.169860,-12.405173,-13.151620,-14.996167,-15.564867,-15.124773,-13.151740,-6.666273,0
48,1.0,2.000000,508.000000,-10.318040,1.120133,-5.598233,-19.458027,-19.864780,-19.841107,-9.824980,...,-24.677293,-13.751307,-13.850100,-16.393133,-17.380913,-20.031173,-18.744973,-16.271420,-9.047427,0
49,1.0,2.000000,568.000000,-11.996347,0.013000,-8.577380,-20.805560,-19.223673,-18.612013,-6.789500,...,-21.147080,-10.814513,-10.239880,-14.415600,-15.481787,-15.696000,-16.079893,-14.557920,-8.897767,0


In [38]:
from sklearn.model_selection import GroupShuffleSplit
n_splits = 20
cv = GroupShuffleSplit(n_splits=n_splits, test_size=0.4)
#df_2 = df_1[(df_1['Value']==1) | (df_1['Value']==9)]
#df_2 = df_2.drop(['O1','O2','OZ','FP1','FP2','F3','FZ','F4','P8','TP8','P7','FC4','FT8','TP7','FCZ','F8','F7','FT7','VEOU','VEOL','HEOR','HEOR'], axis=1)
df_2 = df_1.drop(['VEOU','VEOL','HEOR','HEOL','Value'], axis=1)
df_2 = df_2[df_2['Time'] < 800]
df_2 = df_2[df_2['Time'] > 400]
r_in = np.zeros(n_splits)
r_out = np.zeros(n_splits)
predicted_list = []
actual_list = []
for i,(train, test) in enumerate(cv.split(df_2, groups=df_2['Trial'])):
  #train = df_2.index[train]
  #test = df_2.index[test]
  y = df_2['High Value']
  X = df_2.drop('High Value', axis=1).astype('float')
  rfc.fit(X.iloc[train].values, y.iloc[train])
  predicted = rfc.predict(X.iloc[train])
  
  actual = y.iloc[train]
  
  r_in[i] = np.corrcoef(predicted, actual)[0, 1]
  t_predicted = rfc.predict(X.iloc[test])
  predicted_list.append(t_predicted)
  t_actual = y.iloc[test]
  actual_list.append(t_actual)
  r_out[i] = np.corrcoef(t_predicted, t_actual)[0, 1]
  print(r_in[i], r_out[i])
print("Mean R_out for entire time series = %.3g +/- %.3g" % (r_out.mean(), r_out.std()))


1.0 0.21101009445328323
1.0 0.18460775676119617
1.0 0.10496697825828115
0.9999999999999998 0.19089052525636024
0.9999999999999998 0.05188606333518066
1.0 0.0975561268793804
1.0 0.09606886994330008
1.0 0.11718252571308171
0.9999999999999999 0.09596267831340252
1.0 0.11112539597923797
0.9999999999999999 0.15542150538319424
1.0 0.06544890814732676
1.0 0.1264460239452025
1.0 0.22883896858711184
1.0 0.20196197848215994
1.0 0.1744896978645984
1.0 0.09006249544039002
1.0 0.11434193539268994
0.9999999999999998 0.18510294257795443
0.9999999999999999 0.1424882788696213
Mean R_out for entire time series = 0.137 +/- 0.0498


In [0]:
# This generates a "confusion matrix" showing the number of 
# True Positives, False Positives, True Negatives, and False Negatives, i.e. all of the possible outcomes.  
# We want True Positives (upper left quadrant) and True Negatives (lower right quadrant)
# to be high and the others to be low. 

def report(y, y_predicted):
    conf_mat = confusion_matrix(y, y_predicted)
    report = pd.DataFrame(index=['Actual Low Value', 'Actual High Value'], 
                          columns=['Predicted Low Value', 'Predicted High Value'], 
                          data=conf_mat)
    return report
  
#report(y_pd, y_pd_predicted)

In [62]:

L1 = predicted_list[0]
L1_corr = L1[0::8]
#print(L1_corr)
#print(len(predicted_list[0]))

A1 = actual_list[0]
A1_corr = A1[0::8]
#print(A1_corr)

r = np.corrcoef(L1_corr, A1_corr)[0, 1]
r
report(A1, L1)


,Predicted Low Value,Predicted High Value
Actual Low Value,25,11
Actual High Value,14,23


In [0]:
df_train = df.drop(df.index[train_samp_list])
df_test = df.drop(df.index[test_list])

    
df_x = df_train.drop(['Subject', 'Block', 'Trial'], axis=1)
df_y = df_train.drop(['Subject', 'Block', 'Trial'], axis=1)

In [0]:
y_pd = df_y['Value']
x_pd = df_x

In [0]:
y_pd_predicted = rfc.predict(x_pd)
#formula
#def report(y, y_predicted):
 # conf_mat = confusion_marix(y, y_predicted)
  #report = pd.DataFrame(index=['Value'])
  
print(y_pd_predicted)

[1. 1. 1. ... 3. 3. 3.]
